# Setting up environment

Install packages

In [ ]:
%pip install --upgrade datasets transformers ninja decord timm accelerate
%pip install flash-attn --no-build-isolation

# Using InternVL3 one time

Import packages

In [ ]:
import time
import csv
import logging
import math
import torch
import torchvision.transforms as T
import torch.nn.functional as F
import numpy as np
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForCausalLM, CLIPProcessor, CLIPModel
from huggingface_hub import snapshot_download

Split model

In [ ]:
def split_model(model_path):
    device_map = {}
    world_size = torch.cuda.device_count()
    config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
    num_layers = config.llm_config.num_hidden_layers
    # Since the first GPU will be used for ViT, treat it as half a GPU.
    num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
    num_layers_per_gpu = [num_layers_per_gpu] * world_size
    num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)
    layer_cnt = 0
    for i, num_layer in enumerate(num_layers_per_gpu):
        for j in range(num_layer):
            device_map[f'language_model.model.layers.{layer_cnt}'] = i
            layer_cnt += 1
    device_map['vision_model'] = 0
    device_map['mlp1'] = 0
    device_map['language_model.model.tok_embeddings'] = 0
    device_map['language_model.model.embed_tokens'] = 0
    device_map['language_model.output'] = 0
    device_map['language_model.model.norm'] = 0
    device_map['language_model.model.rotary_emb'] = 0
    device_map['language_model.lm_head'] = 0
    device_map[f'language_model.model.layers.{num_layers - 1}'] = 0

    return device_map

Load dataset and model

In [ ]:
dataset = load_dataset("lmms-lab/AISG_Challenge")
snapshot_download("OpenGVLab/InternVL3-78B", local_dir="./InternVL3-78B")
model = AutoModel.from_pretrained(
    "./InternVL3-78B",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True,
    device_map=split_model('./InternVL3-78B'),
    local_files_only=True).eval()
tokenizer = AutoTokenizer.from_pretrained('./InternVL3-78B', trust_remote_code=True, use_fast=False, local_files_only=True)
text_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-7B-Instruct",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="cuda")
text_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")
generation_config = dict(max_new_tokens=1024, do_sample=False)
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)
clip_model = model = CLIPModel.from_pretrained(
    "openai/clip-vit-base-patch16",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="cuda")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

Download video function

In [ ]:
def retrieve_video(video_id):
    filename = f"{video_id}.mp4"
    video_path = f"./videos/videos/{filename}"
    return video_path

Improve prompt

In [ ]:
def improve_question(question):
    if len(question.split('\n')) <= 1:
        actual_question = question.split('\n')[0]
        
        prompt = f"Generalise the question using only one 5W1H method (Who, What, When, Where, Why, and How). Focus on the main idea of the question instead of specific assumptions. Only return the rewritten question.\nOriginal question: {actual_question}\nRewritten question:"
        messages = [
            {"role": "user", "content": prompt}
        ]
        text = text_tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = text_tokenizer([text], return_tensors="pt").to(text_model.device)
        generated_ids = text_model.generate(
            **model_inputs,
            max_new_tokens=128,
            do_sample=False,
            temperatue=None,
            top_p=None,
            top_k=None
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        response = text_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        
        return response + " " + question
    else:
        return question

Narrow down question scope

In [ ]:
def narrow(question):
    actual_question = question.split('\n')[0]
    
    prompt = f"Break down the question into object, subject, temporal or spatial information in the following format:\nobject, subject, temporal, spatial\nThe question is: {actual_question}."
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = text_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = text_tokenizer([text], return_tensors="pt").to(text_model.device)
    generated_ids = text_model.generate(
        **model_inputs,
        max_new_tokens=256,
        do_sample=False,
        temperatue=None,
        top_p=None,
        top_k=None
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = text_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return response

Build transform

In [ ]:
def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

Find closest aspect ratio

In [ ]:
def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

Dynamic preprocess

In [ ]:
def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

Load video guided with question

In [ ]:
def load_video_question_guided(video_path, question, input_size=448, max_num=1, num_segments=32):
    vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
    max_frame = len(vr)
    step = int(vr.get_avg_fps())

    sampled_indices = list(range(0, max_frame, step))
    all_images = [Image.fromarray(vr[i].asnumpy()).convert("RGB") for i in sampled_indices]
    
    # Encode all frames
    frame_embeddings = []
    for img in all_images:
        inputs = clip_processor(images=img, return_tensors="pt").to("cuda")
        with torch.no_grad():
            img_feat = clip_model.get_image_features(**inputs)
        frame_embeddings.append(img_feat.squeeze(0))
    frame_embeddings = torch.stack(frame_embeddings)

    # Encode question
    text_inputs = clip_processor(text=question, return_tensors="pt").to("cuda")
    with torch.no_grad():
        text_embedding = clip_model.get_text_features(**text_inputs).squeeze(0)

    # Compute similarity
    sims = F.cosine_similarity(frame_embeddings, text_embedding.unsqueeze(0))
    k = min(num_segments, len(sims))
    top_indices = sorted(sims.topk(k).indices.tolist())
    selected_frames = [all_images[i] for i in top_indices]
    '''
    for i in selected_frames:
        i.show()
    '''
    # Your existing preprocessing
    pixel_values_list, num_patches_list = [], []
    transform = build_transform(input_size=input_size)
    for img in selected_frames:
        tiles = dynamic_preprocess(img, image_size=input_size, use_thumbnail=True, max_num=max_num)
        pixel_values = [transform(tile) for tile in tiles]
        pixel_values = torch.stack(pixel_values)
        num_patches_list.append(pixel_values.shape[0])
        pixel_values_list.append(pixel_values)
    pixel_values = torch.cat(pixel_values_list)
    return pixel_values, num_patches_list

Process sample

In [ ]:
def process_test_case(example):
    video_id = example["video_id"]
    question = example["question"]
    question_prompt = example["question_prompt"]
    
    video_path = retrieve_video(video_id)
    improved_question = improve_question(question)

    pixel_values, num_patches_list = load_video_question_guided(video_path, question.split('\n')[0], num_segments=32, max_num=1)
    pixel_values = pixel_values.to(torch.bfloat16).cuda()
    video_prefix = ''.join([f'Frame{i+1}: <image>\n' for i in range(len(num_patches_list))])

    prompt = video_prefix + f"\nFind the following information from the video:\n{narrow(question)}"
    response, history = model.chat(tokenizer, pixel_values, prompt, generation_config, num_patches_list=num_patches_list, history=None, return_history=True)
    '''
    print(f"1st Prompt:\n{prompt}")
    print(f"1st Response:\n{response}")
    '''
    prompt = video_prefix + f"\n{improved_question}\n{question_prompt}"
    response, history = model.chat(tokenizer, pixel_values, prompt, generation_config, num_patches_list=num_patches_list, history=history, return_history=True)
    '''
    print(f"2nd Prompt:\n{prompt}")
    print(f"2nd Response:\n{response}")
    print(f"Video URL: {example['youtube_url']}")
    '''
    return example['qid'], response

Run test cases

In [ ]:
start_time = time.time()
output_file = "./results.csv"
start_processing = False
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
with open(output_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["qid", "pred"])
    '''
    sample = dataset['test'].filter(lambda x: x['qid'] == '0340-0')[0]
    qid, pred = process_test_case(sample)
    writer.writerow([qid, pred])
    '''
    for sample in dataset['test']:
        if not start_processing:
            if sample['qid'] == "0008-0":
                start_processing = True
            else:
                continue
        logging.debug(f"processing qid {sample['qid']}")
        qid, pred = process_test_case(sample)
        writer.writerow([qid, pred])
    
    end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")